In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
import tarfile 
import scipy.io
import cv2
import os

%matplotlib inline 

In [2]:
# defining feature extractor that we want to use
extractor = cv2.xfeatures2d.SIFT_create()

def features(image, extractor):
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return keypoints, descriptors

In [3]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters = 500)
X_des_train = kmeans.fit(descriptor_list)

preprocessed_image = []
for image in images:
    image = gray(image)
    keypoint, descriptor = features(image, extractor)
    if (descriptor is not None):
    histogram = build_histogram(descriptor, kmeans)
    preprocessed_image.append(histogram)

NameError: name 'descriptor_list' is not defined